# Sprint 3 Data Science Report
## NFT Ticketing Platform - Fraud Detection Model Evaluation

**Report Date**: 2025-11-28  
**Model Version**: v1.2.3  
**Platform Scale**: 50k-200k daily events


## 1. Data Preparation

### Load and Explore Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Load data (if available, otherwise use sample)
try:
    df = pd.read_csv('../demos/data/sample_transactions.csv')
except:
    # Generate sample data
    np.random.seed(42)
    n = 1000
    df = pd.DataFrame({
        'transaction_id': [f'txn_{i:06d}' for i in range(n)],
        'wallet_address': [f'0x{i%100:040x}' for i in range(n)],
        'txn_velocity_1h': np.random.poisson(2, n),
        'wallet_age_days': np.random.exponential(30, n),
        'avg_ticket_hold_time': np.random.normal(48, 12, n),
        'event_popularity_score': np.random.beta(2, 5, n),
        'price_deviation_ratio': np.random.normal(0, 0.3, n),
        'cross_event_attendance': np.random.poisson(2, n),
        'geo_velocity_flag': np.random.binomial(1, 0.05, n),
        'payment_method_diversity': np.random.poisson(1, n) + 1,
        'social_graph_centrality': np.random.beta(1, 2, n),
        'time_to_first_resale': np.random.exponential(720, n),
        'is_fraud': (np.random.random(n) < 0.02).astype(int)
    })

print(f"Dataset shape: {df.shape}")
print(f"Fraud rate: {df['is_fraud'].mean():.2%}")
df.head()


### Feature List

**10 Core Features:**
1. `txn_velocity_1h` - Transaction velocity in 1 hour
2. `wallet_age_days` - Wallet age in days
3. `avg_ticket_hold_time` - Average ticket hold time
4. `event_popularity_score` - Event popularity score
5. `price_deviation_ratio` - Price deviation from floor
6. `cross_event_attendance` - Cross-event attendance count
7. `geo_velocity_flag` - Geographic velocity flag
8. `payment_method_diversity` - Payment method diversity
9. `social_graph_centrality` - Social graph centrality
10. `time_to_first_resale` - Time to first resale (minutes)


In [ ]:
# Feature statistics
feature_cols = [
    'txn_velocity_1h',
    'wallet_age_days',
    'avg_ticket_hold_time',
    'event_popularity_score',
    'price_deviation_ratio',
    'cross_event_attendance',
    'geo_velocity_flag',
    'payment_method_diversity',
    'social_graph_centrality',
    'time_to_first_resale'
]

df[feature_cols].describe()


## 2. Model Training

### Prepare Training Data


In [ ]:
# Prepare features and target
X = df[feature_cols].fillna(0)
y = df['is_fraud'].astype(int)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Train fraud rate: {y_train.mean():.2%}")
print(f"Test fraud rate: {y_test.mean():.2%}")


In [ ]:
# Train XGBoost model
from xgboost import XGBClassifier

fraud_count = y_train.sum()
legit_count = len(y_train) - fraud_count
scale_pos_weight = legit_count / fraud_count if fraud_count > 0 else 1.0

model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    scale_pos_weight=scale_pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='aucpr',
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
print("✅ Model training complete")


## 3. Performance Metrics

### Evaluate Model


In [ ]:
# Generate predictions
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.65).astype(int)

# Calculate metrics
auc_roc = roc_auc_score(y_test, y_pred_proba)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)

print(f"AUC-ROC: {auc_roc:.3f}")
print(f"AUC-PR:  {auc_pr:.3f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Legit', 'Fraud']))


## 4. Evaluation Discussion

### Assumptions
- Historical fraud patterns remain stable
- Fraudsters don't adapt faster than retraining cycle
- On-chain data is ground truth

### Potential Biases
- **Geographic bias**: Model trained on US/EU events
- **Temporal bias**: Holiday/event seasonality not fully captured
- **Labeling bias**: Manual fraud labels may miss sophisticated attacks

### Mitigation Strategies
- Weekly drift monitoring
- Quarterly model retraining
- Human-in-the-loop review for borderline cases



In [ ]:
# Feature importance visualization
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("\nTop 5 Features:")
print(feature_importance.head())
